In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

import cv2
import skimage
import IPython.display
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from glob import glob

from collections import OrderedDict
import torch
import gc
import cuml

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
!ls ../input/hm-emb-clip-vit32

In [ ]:
art = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/articles.csv', dtype='str')
art['detail_desc'] = art['detail_desc'].apply(lambda x: x.lower() if x==x else 'no description' )
art['path'] = art['article_id'].apply(lambda x: '../input/h-and-m-personalized-fashion-recommendations/images/' + x[:3] + '/' + x + '.jpg')
art.head(2)

In [ ]:
art.loc[art['department_no']=='8398' , 'department_no' ] = '8316'
art.loc[art['department_no']=='1620' , 'department_no' ] = '1522'
art.loc[art['department_no']=='1727' , 'department_no' ] = '1522'
art.loc[art['department_no']=='3931' , 'department_no' ] = '3932'
art.loc[art['department_no']=='3938' , 'department_no' ] = '3936'
art.loc[art['department_no']=='3939' , 'department_no' ] = '3936'
art.loc[art['department_no']=='3943' , 'department_no' ] = '5683'
art.loc[art['department_no']=='5787' , 'department_no' ] = '5831'
art.loc[art['department_no']=='5454' , 'department_no' ] = '5631'
art.loc[art['department_no']=='7659' , 'department_no' ] = '7658'
art.loc[art['department_no']=='2930' , 'department_no' ] = '2932'
art.loc[art['department_no']=='7854' , 'department_no' ] = '7848'
art.loc[art['department_no']=='7819' , 'department_no' ] = '7917'
art.loc[art['department_no']=='3510' , 'department_no' ] = '1710'
art.loc[art['department_no']=='7989' , 'department_no' ] = '7988'
art.loc[art['department_no']=='2932' , 'department_no' ] = '7814'
art.loc[art['department_no']=='8564' , 'department_no' ] = '8563'

In [ ]:
art.columns

In [ ]:
art['index_name'].value_counts().tail(50)

In [ ]:
art['index_group_name'].value_counts().tail(50)

In [ ]:
art.loc[art['section_name']=='Ladies Other' , 'section_name'] = 'Womens Everyday Collection'
art['section_name'].value_counts()

In [ ]:
img = cv2.imread(art.path.values[0])
print(img.shape)
img = cv2.resize(img, (256,256))
plt.imshow(img)

In [ ]:
EMB  = np.load('../input/hm-emb-clip-vit32/text0-emb.npy')
EMB += np.load('../input/hm-emb-clip-vit32/text1-emb.npy')
EMB += np.load('../input/hm-emb-clip-vit32/text2-emb.npy')
EMB += np.load('../input/hm-emb-clip-vit32/text3-emb.npy')
EMB += np.load('../input/hm-emb-clip-vit32/text4-emb.npy')
EMB = 20 * np.load('../input/hm-emb-clip-vit32/image-emb.npy')
EMB /= 25.
EMB.shape

In [ ]:
art.columns

In [ ]:
art.nunique()

In [ ]:
dt = art.groupby(['department_no','index_code'])['article_id'].agg('count').reset_index()
dt[dt['article_id']<=15]

In [ ]:
dt = art.groupby(['department_no','index_code'])['article_id'].agg('count').reset_index()
dt

In [ ]:
art.columns

In [ ]:
n_neighbors = 5
INDICES = np.zeros( (art.shape[0], n_neighbors), dtype='int32' )
ind_ref = np.arange(art.shape[0])

dt = art.groupby(['department_no','index_code'])['article_id'].agg('count').reset_index()
for n in range(dt.shape[0]):
    var0 = dt['department_no'].values[n]
    var1 = dt['index_code'].values[n]
    print(var0, var1)
    ind = np.where( (art.department_no.values == var0)&(art.index_code.values == var1) )[0]
    
    nn = cuml.neighbors.NearestNeighbors(n_neighbors=n_neighbors)
    nn.fit(EMB[ind])
    distances, indices = nn.kneighbors(EMB[ind])
    ind = ind_ref[ind]
    indices = ind[indices]
    INDICES[ind] = indices.astype('int32')
gc.collect()

In [ ]:
INDICES[:15]

In [ ]:
var = INDICES[126,:]
for i in range(4):
    img = cv2.imread(art.path.values[var[i]])
    plt.imshow(img)
    plt.show()

In [ ]:
var = INDICES[111,:]
for i in range(4):
    img = cv2.imread(art.path.values[var[i]])
    plt.imshow(img)
    plt.show()

In [ ]:
var = INDICES[1125,:]
for i in range(4):
    img = cv2.imread(art.path.values[var[i]])
    plt.imshow(img)
    plt.show()

In [ ]:
var = INDICES[1135,:]
for i in range(4):
    img = cv2.imread(art.path.values[var[i]])
    plt.imshow(img)
    plt.show()

In [ ]:
var = INDICES[1225,:]
for i in range(4):
    img = cv2.imread(art.path.values[var[i]])
    plt.imshow(img)
    plt.show()

In [ ]:
var = INDICES[1525,:]
for i in range(4):
    img = cv2.imread(art.path.values[var[i]])
    plt.imshow(img)
    plt.show()

In [ ]:
var = INDICES[3125,:]
for i in range(1, 4):
    img = cv2.imread(art.path.values[var[i]])
    plt.imshow(img)
    plt.show()

In [ ]:
for i in range(1, 5):
    art['nn'+str(i-1)] = art['article_id'].values[ INDICES[:,i] ]
    
art

In [ ]:
art.to_parquet('articles_nn.parquet')